import os
os.environ['PATH']
stream = os.popen('owltools --help')
output = stream.read()
output

In [ ]:
import pandas as pd
import os
from rdflib import Graph
from urllib.request import urlopen
import subprocess
import time
from IPython.display import Markdown as md
from IPython.display import HTML

# use ontobio!!
# pd.set_option('display.notebook_repr_html',False)
# https://github.com/jupyter/nbconvert/issues/946

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

# Mondo analysis

from ontobio.ontol_factory import OntologyFactory
mondo_file="/Users/matentzn/ws/mondo/src/ontology/mondo-qc.obo"
ofa = OntologyFactory()
ont = ofa.create(mondo_file)
ont.get_roots()[:3]

In [ ]:
#sparql_dir_remote="https://raw.githubusercontent.com/monarch-initiative/mondo/no-subclass-between-genetic-disease/src/sparql/"
#mondo = "mondo-qc.owl"
#sparql_dir_local="../sparql/"
reports_dir_local="./"
warnings ="single-child no-subclass-between-genetic-disease related-exact-synonym excluded-subsumption-is-inferred".split(" ")
warning_reports = [warn+"-warning" for warn in warnings] 
NUMBER_ERRORS_SHOWN=10

mondo="http://purl.obolibrary.org/obo/mondo.owl"
g = Graph()
g.parse("../mondo-qc.owl", format="xml")

In [ ]:
def sparql_path(sparql):
    return os.path.join(sparql_dir_local,sparql+".sparql")

def report_path(report):
    return os.path.join(reports_dir_local,report+".tsv")

def get_report(name):
    try:
        return pd.read_csv(report_path(name),sep="\t")
    except pd.errors.EmptyDataError:
        print("WARNING! ", name, " is empty and has been skipped.")
    return pd.DataFrame(columns=['EMPTY'])
        
def load_from_uri(uri):
    query = urlopen(sparql_iri).read().decode('utf-8')
    return query


def load_from_local_file(file_path):
    with open(file_path, 'r') as file:
        data = file.read()
    return data


## Robot report analysis

In [ ]:
df = get_report("obo-report")
# Bug in ROBOT, wrong number of columns:
df.columns=['Rule Name', 'Subject', 'Property', 'Value','Trash']
df['Level'] = df.index
df.reset_index(level=0, inplace=True)
del df['index']
del df['Trash']

### Overview of errors

This breaks down the errors in the generic ROBOT report.

In [ ]:
df=df[df['Subject'].str.startswith('MONDO')]
df[['Level','Rule Name','Value']].groupby(['Level','Rule Name']).count()

In [ ]:
for rule in df['Rule Name'].unique():
    display(md("#### %s" % rule))
    dfd = df[df['Rule Name']==rule].head(NUMBER_ERRORS_SHOWN)
    display(HTML(dfd.to_html(index=False)))


# Revealed a more complex issue we need to sort first: https://github.com/monarch-initiative/mondo/issues/1979
# https://github.com/ontodev/robot/issues/733 # Ignore 0021108 0045024 until ticket resolved.

# Other checks analyses

In [ ]:
for r in warning_reports:
    df_r = get_report(r)
    if (len(df_r)>0):
        display(md("#### %s" % r))
        df_rd = df_r.head(NUMBER_ERRORS_SHOWN).copy()
        display(HTML(df_rd.to_html(index=False)))

## Mondo Statistics